# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [13]:
import networkx as nx
G = nx.Graph()
usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [14]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 2
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [15]:
G = nx.DiGraph(G)
uniform_capacity = 10
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [16]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    #Check if necessary condition is met: demand = supply
    sum_demand=0
    sum_supply=0
    for node in G.nodes():
        if G.node[node]['demand']>=0:
            sum_demand=sum_demand + G.node[node]['demand']
        else:
            sum_supply=sum_supply + G.node[node]['demand']

    if (sum_demand+sum_supply)!=0:
        raise nx.NetworkXUnfeasible()
    
    #Graph reduction
    source='S'
    sink='T'
    for state in G.nodes():
        if G.node[state]['demand']<0:         
            G.add_edge(source,state)
            G.edge[source][state]['capacity'] = -G.node[state]['demand']
        else:
            G.add_edge(state,sink)
            G.edge[state][sink]['capacity']=G.node[state]['demand']
            
    #Apply Edmonds Karp 
#     R=edmonds_karp(G,source,sink)
#     if R.graph['flow_value']!=sum_demand:
#         return "NetworkXUnfeasible"
        
    #Apply Max_flow to get maximum flow
    F=nx.maximum_flow(G,source,sink)
    
    #Check if max flow is also feasible flow
    if F[0]!=sum_demand:
        return nx.NetworkXUnfeasible()
    
    #Remove 'S' and 'T' nodes from the graph and dictionary flow
    G.remove_node('S')
    G.remove_node('T')
    flow=F[1]
    del flow['S']
    del flow['T']
    for node in G.nodes():
        if 'T' in flow[node]:
            del flow[node]['T']
        if 'S' in flow[node]:
            del flow[node]['S']
    return flow    
    # TODO: Implement the function.

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [17]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    net_flow={}
    for node1 in G.nodes():
        sum1=0
        sum2=0
        for node2 in G.neighbors(node1):
            if node1 in flow[node2]:
                sum1=sum1+flow[node2][node1]
            if node2 in flow[node1]:
                sum2=sum2+flow[node1][node2]
        net_flow[node1]=sum1-sum2
    
    return net_flow
    # TODO: Implement the function.

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [18]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

NetworkXUnfeasible: 